In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import keras
from keras.datasets import cifar10  # dataset to be used for the assignment
from keras.utils import np_utils
import tensorflow as tf
from tensorflow.keras import datasets, layers, models
from keras.layers import *
from keras import backend as K
import os
from keras import regularizers
import tensorflow.python.keras
from keras.preprocessing.image import ImageDataGenerator

In [ ]:
# TO CHECK IF ANY GPUS ARE AVAILABLE

from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 11582550410570162829
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 11345264640
locality {
  bus_id: 1
  links {
  }
}
incarnation: 16992684562107903143
physical_device_desc: "device: 0, name: Tesla K80, pci bus id: 0000:00:04.0, compute capability: 3.7"
]


In [ ]:
def myGetModel(data):
    weight_decay = 1e-4
    model = models.Sequential()

    # 1
    model.add(layers.Conv2D(32, (3, 3), padding='same', activation='relu',
                     kernel_regularizer=regularizers.l2(weight_decay),
                     input_shape=data.input_dim))
    model.add(layers.BatchNormalization())

    # 2
    model.add(layers.Conv2D(32, (3, 3), padding='same',
                     kernel_regularizer=regularizers.l2(weight_decay),
                     activation='relu'))
    model.add(layers.BatchNormalization())
    model.add(layers.MaxPooling2D(pool_size=(2, 2)))
    model.add(layers.Dropout(0.2))

    # 3
    model.add(Conv2D(64, (3, 3), padding='same',
                     kernel_regularizer=regularizers.l2(weight_decay),
                     activation='relu'))
    model.add(BatchNormalization())

    # 4
    model.add(Conv2D(64, (3, 3), padding='same',
                     kernel_regularizer=regularizers.l2(weight_decay),
                     activation='relu'))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.3))

    # 5
    # model.add(Conv2D(128, (3, 3), padding='same',
    #                  kernel_regularizer=regularizers.l2(weight_decay),
    #                  activation='relu'))
    # model.add(BatchNormalization())
    model.add(Conv2D(64, (3, 3),
                     padding='same',
                    #  kernel_regularizer=regularizers.l2(weight_decay),
                     activation='relu'))

    # 6
    # model.add(Conv2D(128, (3, 3),
    #                  padding='same',
    #                  kernel_regularizer=regularizers.l2(weight_decay),
    #                  activation='relu'))

    # NOTE: Some of the layers have been removed to reduce computational time.
    # The obtained accuracy exceeds the par function without these layers. However,
    # these can uncommented to further improve the results.
    
    model.add(Conv2D(64, (3, 3),
                     padding='same',
                    #  kernel_regularizer=regularizers.l2(weight_decay),
                     activation='relu'))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.4))
    model.add(Flatten())
    model.add(Dense(data.num_classes, activation='softmax'))

    opt_rms = tensorflow.keras.optimizers.RMSprop(learning_rate=0.001,
                                                  decay=1e-6)
    
    # NOTE: rms prop works much better 
    model.compile(loss='categorical_crossentropy', optimizer=opt_rms,
                  metrics=['accuracy'])

    # lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
    #     initial_learning_rate=0.1,
    #     decay_rate=1e-6,
    #     decay_steps=0.1)
    # optimizer = tf.keras.optimizers.SGD(learning_rate=lr_schedule)  # decay_steps=0.1)
    # model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

    return model

In [ ]:
# NOTE

# from google.colab import drive # UNCOMMENT WHEN RUNNING ON GOOGLE COLAB
# drive.mount('/content/gdrive') # UNCOMMENT WHEN RUNNING ON GOOGLE COLAB

def myFitModel(model, data):
    # checkpoint_path = os.path.dirname(os.path.realpath(__file__))  # UNCOMMENT WHEN RUNNING ON WINDOWS/MAC
    checkpoint_path = "/content/gdrive/MyDrive/Colab Notebooks" # UNCOMMENT WHEN RUNNING ON GOOGLE COLAB
    keras_callbacks = [
        tf.keras.callbacks.EarlyStopping(monitor='val_loss',
                                         patience=20, mode='min', min_delta=0.01,
                                         restore_best_weights=True),
        tf.keras.callbacks.ModelCheckpoint(checkpoint_path,
                                           monitor='val_loss',
                                           save_best_only=True, mode='min')
    ]
    # print('keras_callbacks', keras_callbacks)
    
    batch_size = 64
    steps_per_epoch = len(data.x_train)//batch_size
    validation_steps_per_epoch = len(data.x_valid)//batch_size
    history = model.fit(data.x_train, data.y_train, 
                        batch_size=batch_size, epochs=10,
                        steps_per_epoch=steps_per_epoch,
                        validation_data=(data.x_valid, data.y_valid), 
                        validation_steps=validation_steps_per_epoch,
                        verbose=2,
                        callbacks=keras_callbacks)

    return model

In [ ]:
def runImageClassification(seed=7):
    # Fetch data. You may need to be connected to the internet the first time this is done.
    # After the first time, it should be available in your system. On the off chance this
    # is not the case on your system and you find yourself repeatedly downloading the data, 
    # you should change this code so you can load the data once and pass it to this function. 
    print("Preparing data...")
    data = CIFAR(seed)
    print(data.y_valid_raw.shape)

    # Create model
    print("Creating model...")
    model = myGetModel(data)

    # Fit model
    print("Fitting model...")
    model = myFitModel(model, data)

    # Evaluate on test data
    print("Evaluating model...")
    score = model.evaluate(data.x_test, data.y_test, verbose=0)
    print('Test accuracy:', score[1])

runImageClassification()

Preparing data...
(9000, 1)
Creating model...
Fitting model...
Epoch 1/10
703/703 - 18s - loss: 1.6827 - accuracy: 0.4595 - val_loss: 1.2456 - val_accuracy: 0.5644
INFO:tensorflow:Assets written to: /content/gdrive/MyDrive/Colab Notebooks/assets
Epoch 2/10
703/703 - 16s - loss: 1.0512 - accuracy: 0.6404 - val_loss: 0.9385 - val_accuracy: 0.6823
INFO:tensorflow:Assets written to: /content/gdrive/MyDrive/Colab Notebooks/assets
Epoch 3/10
703/703 - 16s - loss: 0.8759 - accuracy: 0.7023 - val_loss: 0.8353 - val_accuracy: 0.7171
INFO:tensorflow:Assets written to: /content/gdrive/MyDrive/Colab Notebooks/assets
Epoch 4/10
703/703 - 16s - loss: 0.7765 - accuracy: 0.7378 - val_loss: 1.5428 - val_accuracy: 0.5777
Epoch 5/10
703/703 - 16s - loss: 0.7113 - accuracy: 0.7615 - val_loss: 0.7411 - val_accuracy: 0.7584
INFO:tensorflow:Assets written to: /content/gdrive/MyDrive/Colab Notebooks/assets
Epoch 6/10
703/703 - 16s - loss: 0.6635 - accuracy: 0.7791 - val_loss: 0.7580 - val_accuracy: 0.7517
Epo

In [ ]:
class CIFAR:
    def __init__(self, seed=0):
        # Get and split data
        data = self.__getData(seed)
        self.x_train_raw = data[0][0]
        self.y_train_raw = data[0][1]
        self.x_valid_raw = data[1][0]
        self.y_valid_raw = data[1][1]
        self.x_test_raw = data[2][0]
        self.y_test_raw = data[2][1]
        # Record input/output dimensions
        self.num_classes=10
        self.input_dim = self.x_train_raw.shape[1:]
        # Convert data
        self.y_train = np_utils.to_categorical(self.y_train_raw, self.num_classes)
        self.y_valid = np_utils.to_categorical(self.y_valid_raw, self.num_classes)
        self.y_test = np_utils.to_categorical(self.y_test_raw, self.num_classes)
        self.x_train = self.x_train_raw.astype('float32')
        self.x_valid = self.x_valid_raw.astype('float32')
        self.x_test = self.x_test_raw.astype('float32')
        self.x_train  /= 255
        self.x_valid  /= 255
        self.x_test /= 255
        # Class names
        self.class_names = ['airplane', 'automobile', 'bird', 'cat', 'deer',
                            'dog', 'frog', 'horse', 'ship', 'truck']

    def __getData(self,seed=0):
        (x_train, y_train), (x_test, y_test) = cifar10.load_data()
        return self.__shuffleData(x_train,y_train,x_test,y_test,seed)
    
    def __shuffleData (self,x_train,y_train,x_test,y_test,seed=0):
        tr_perc = .75
        va_perc = .15
        x=np.concatenate((x_train, x_test))
        y=np.concatenate((y_train, y_test))
        np.random.seed(seed)
        np.random.shuffle(x)
        np.random.seed(seed)
        np.random.shuffle(y)
        indices = np.random.permutation(len(x))
        tr = round(len(x)*tr_perc)
        va = round(len(x)*va_perc)
        self.tr_indices = indices[0:tr]
        self.va_indices = indices[tr:(tr+va)]
        self.te_indices = indices[(tr+va):len(x)]
        x_tr=x[self.tr_indices ,]
        x_va=x[self.va_indices ,]
        x_te=x[self.te_indices ,]
        y_tr=y[self.tr_indices ,]
        y_va=y[self.va_indices ,]
        y_te=y[self.te_indices,]
        return ((x_tr,y_tr),(x_va,y_va),(x_te,y_te))

    # Print 25 random figures from the validation data
    def showImages(self):
        images=self.x_valid_raw
        labels=self.y_valid_raw
        class_names= ['airplane', 'automobile', 'bird', 'cat', 'deer',
                       'dog', 'frog', 'horse', 'ship', 'truck']
        plt.figure(figsize=(10, 10))
        indices=np.random.randint(0, images.shape[0], 25)
        for i in range(25):
            plt.subplot(5, 5, i+1)
            plt.xticks([])
            plt.yticks([])
            plt.grid(False)
            plt.imshow(images[indices[i]], cmap=plt.cm.binary)
            # The CIFAR labels happen to be arrays, 
            # which is why we need the extra index
            plt.xlabel(class_names[labels[indices[i]][0]])
        plt.show()